This notebook generates detector plane images (dpis) of glitches caused by heat pipe-induced count rate increases in the SWIFT Burst Alert Telescope (BAT). 

Glitch events are pulled from GUANO and the event data is centered around the glitch using a 64ms time window. The DPI is constructed from the DETX and DETY coordinates of each detected event.

Two resolutions are available, corresponding to two support vector machine models:

"original" : full detector plane resolution (286 x 173)

"lowRES" : binned resolution (~ 16 x 16)

GUANO dump, if too large, will claim "out of index" and not run. ~ 1.5 years is the max i have found, needs adjusting to get desired amount of dpi saves.

In [22]:
resolution = "original"  # Choose between "original" or "lowRES"
total = 300  # Number of dpi files to save

# Enter beginning and ending dates for GUANO pull in "yyyy-mm-dd" format
begin = "2024-01-01"    # default is 2024-01-01 to 2025-01-01
end = "2024-06-01"   

In [23]:
from swifttools.swift_too import Data, GUANO
from astropy.table import Table
import matplotlib.pyplot as plt
import numpy as np
import os

### helper functions

In [24]:
# gets 16ms bin of "bad time intervals" where glitch happens
def get_btis_for_glitches(evdata, tstart, tstop, tbin_size=.016):
    all_bad_twinds = []

    for start, stop in zip(tstart, tstop):
        bins = np.arange(start, stop + tbin_size / 2.0, tbin_size)
        ebl = evdata["ENERGY"] <= 25.0
        ebl2 = evdata["ENERGY"] > 50.0

        h = np.histogram(evdata["TIME"][ebl], bins=bins)[0]
        h2 = np.histogram(evdata["TIME"][ebl2], bins=bins)[0]
        
        stds = (h - np.mean(h)) / np.std(h)
        stds2 = (h2 - np.mean(h2)) / np.std(h2)
        
        bl_lowE_highSNR = stds > 10.0
        bl_highE_lowSNR = (stds / np.abs(stds2)) > 3
        bl_bad = bl_highE_lowSNR & bl_lowE_highSNR
    
        bad_twinds = []
        for i in range(np.sum(bl_bad)):
            t0 = bins[:-1][bl_bad][i]
            t1 = t0 + tbin_size
            bad_twind = (t0 - .024, t1 + .024)
            # 24ms before plus 16ms interval plus 24ms after for 64ms total
            bad_twinds.append(bad_twind)      

        if bad_twinds:
            all_bad_twinds.extend(bad_twinds)
    
    return all_bad_twinds


def det2dpi_sands_ebins(tab, ebins, weights=None):

    detxs_by_sand0 = np.arange(0, 286 - 15, 18)
    detxs_by_sand1 = detxs_by_sand0 + 15
    xbins = np.append(detxs_by_sand0, [detxs_by_sand1[-1]])

    detys_by_sand0 = np.arange(0, 173 - 7, 11)
    detys_by_sand1 = detys_by_sand0 + 7
    ybins = np.append(detys_by_sand0, [detys_by_sand1[-1]])
    
    sand_dpis = np.swapaxes(
        np.histogramdd([tab["DETX"], tab["DETY"], tab['ENERGY']], bins=[xbins, ybins, ebins], weights=weights)[
            0
        ],
        0,
        1,
    )

    return sand_dpis


def save_dpi(dpi, folder, prefix, saved, i, obsid):
    dpi_filename = f"{prefix}{saved + 1}_window_{i + 1}.npy"
    dpi_filepath = os.path.join(folder, dpi_filename)
    print(f"Saved DPI data for ObsID {obsid}, as {dpi_filename} in {folder} folder.")
    np.save(dpi_filepath, dpi)
    return dpi_filename

### main

In [25]:
def NEWmain(resolution = "original", total = 300, begin="2024-01-01", end="2025-01-01"):   # default values
    assert resolution in ["original", "lowRES"], "Resolution must be 'original' or 'lowRES'"
    
    glitchdpi_folder = "GLITCHdpi" if resolution == "original" else "GLITCHdpi_lowRES"
    os.makedirs(glitchdpi_folder, exist_ok=True)
    
    scratchbase = os.path.expanduser("~/scratch")
    prefix = f"GLITCHdpi{'lowRES' if resolution == 'lowRES' else ''}_"

    existing_files = [f for f in os.listdir(glitchdpi_folder) if f.startswith(prefix) and f.endswith(".npy")]
    
    saved = len(existing_files)  # Start numbering from the next available file
    
    guano = GUANO(begin=begin, end=end)
    processedobsids = set()
    index = 0

    if saved == total:
        print(f"{saved} files already in {glitchdpi_folder}. Done.")
        return
    
    while saved < total:

        obsid = guano[index].obsnum
        
        print(f"\n\tIndex {index}")
        print(f"Glitch {saved}")
        print(f"Processing ObsID: {obsid}")

        obsid_folder = os.path.join(scratchbase, str(obsid), "bat", "event")
        obsname = os.path.join(scratchbase, str(obsid))
        fname = f"sw{obsid}bevshpo_uf.evt.gz"
        event_url = os.path.join(obsid_folder, fname)
                   
        
        if os.path.exists(event_url) or os.path.exists(obsname):
            print(f"Skipping ObsID {obsid} as event file already exists in directory.")
            processedobsids.add(obsid)
            index += 1
            continue
        
        os.makedirs(obsid_folder, exist_ok=True)

        try:
            print(f"Downloading {obsid} from GUANO.") 
            # Download event file from GUANO
            data = Data(obsid=obsid, bat=True, outdir=scratchbase, clobber=True)
            print(f"Downloaded event file to: {event_url}")
        except Exception as e:
            print(f"Error downloading event file for ObsID {obsid}: {e}. Skipping.")
            processedobsids.add(obsid)
            index += 1
            continue
        
                
        try:
            evtable = Table.read(event_url)
            event_GTI = Table.read(event_url, hdu='GTI')
        except Exception as e:
            print(f"Error reading event file for ObsID {obsid}: {e}. Skipping.")
            processedobsids.add(obsid) 
            index += 1
            continue

        tstart = event_GTI["START"].data
        tstop = event_GTI["STOP"].data

        glitch_twinds = get_btis_for_glitches(evtable, tstart, tstop)

        if not glitch_twinds:
            print(f"No glitches detected for ObsID {obsid}. Skipping.")
            processedobsids.add(obsid)
            index += 1
            continue


        # Process each glitch window
        for i, (t_min, t_max) in enumerate(glitch_twinds):
            bl = (evtable['TIME'] >= t_min) & (evtable['TIME'] <= t_max)
            binned_events = evtable[bl]

            if len(binned_events) == 0:
                continue

            if resolution == "original":
                xbins = np.arange(286 + 1) - 0.5
                ybins = np.arange(173 + 1) - 0.5                
                dpi = np.histogram2d(binned_events['DETX'], binned_events['DETY'], bins=[xbins, ybins])[0]
       
            else:
                ebins = [15, 50, 350]
                dpi = det2dpi_sands_ebins(binned_events, ebins)

            plt.close()
    
            if np.any(dpi):
                dpi_filename = save_dpi(dpi, glitchdpi_folder, prefix, saved, i, obsid)
                saved += 1
            
                if saved >= total:
                    print(f"Done. {total} GLITCH DPIs for {prefix} saved.")
                    return
    
            
        processedobsids.add(obsid)
        index += 1

In [26]:
NEWmain(resolution=resolution, total=total, begin=begin, end=end)


	Index 0
Glitch 277
Processing ObsID: 00090372005
Skipping ObsID 00090372005 as event file already exists in directory.

	Index 1
Glitch 277
Processing ObsID: 00016643004
Skipping ObsID 00016643004 as event file already exists in directory.

	Index 2
Glitch 277
Processing ObsID: 00013509206
Skipping ObsID 00013509206 as event file already exists in directory.

	Index 3
Glitch 277
Processing ObsID: 00030846092
Skipping ObsID 00030846092 as event file already exists in directory.

	Index 4
Glitch 277
Processing ObsID: 00092436006
Skipping ObsID 00092436006 as event file already exists in directory.

	Index 5
Glitch 277
Processing ObsID: 00016642008
Skipping ObsID 00016642008 as event file already exists in directory.

	Index 6
Glitch 277
Processing ObsID: 00097137226
Skipping ObsID 00097137226 as event file already exists in directory.

	Index 7
Glitch 277
Processing ObsID: 00080354007
Skipping ObsID 00080354007 as event file already exists in directory.

	Index 8
Glitch 277
Processing 

KeyboardInterrupt: 